### Monash Data Analytical Bootcamp - Project 2 Assignment 

#### Project Team Members:
*   Megan Greenhill 
*   Hesh Kuruppuge
*   Mike Murphy 

The John Hopkins University (JHU) Covid data extract component of this assignment is based on the tutorial by B Chen 
and is used with his permission. The tutorial is located at the following link. 

*    https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6 
    

This assignment uses the following Covid19 CSV files which are updated daily and published by John Hopkins University. 

*    time_series_covid19_confirmed_global.csv................confirmed global Covid_19 cases
*    time_series_covid19_deaths_global.csv...................confirmed global Covid_19 deaths
*    time_series_covid19_recovered_global.csv................confirmed global Covid_19 recovered 

It uses the following World Covid Vaccination dataset published by Our World in Data. 

*    https://ourworldindata.org/covid-vaccinations 

It uses the following World Population dataset published by Our World in Data. 

*    https://www.worldometers.info/world-population/population-by-country/ 



In [1]:
#   import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
from scipy.stats import linregress
from urllib.error import HTTPError
import numpy as np
import wget
import time
from datetime import datetime
from sqlalchemy import create_engine
# from pandas_profiling import ProfileReport

#   the project uses urls / wget downloads in place of API calls are they are not available for the datasets needed

# url of the raw csv dataset
urls = [
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
]
[wget.download(url) for url in urls]



['time_series_covid19_confirmed_global (1).csv',
 'time_series_covid19_deaths_global (1).csv',
 'time_series_covid19_recovered_global (1).csv']

In [2]:
#   create dataframes from csv's

confirmed_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
    
deaths_df = pd.read_csv('time_series_covid19_deaths_global.csv')
    
recovered_df = pd.read_csv('time_series_covid19_recovered_global.csv')

confirmed_df


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/7/22,3/8/22,3/9/22,3/10/22,3/11/22,3/12/22,3/13/22,3/14/22,3/15/22,3/16/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,175000,175353,175525,175893,175974,176039,176201,176409,176571,176743
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,272250,272337,272412,272479,272552,272621,272663,272689,272711,272804
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,265297,265323,265346,265366,265391,265410,265432,265457,265478,265496
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,38620,38710,38794,38794,38794,38794,38794,38794,38794,38794
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,98806,98829,98855,98855,98855,98909,98927,98931,98956,98985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,651700,652298,652763,652966,653293,653293,653293,653854,654068,654068
280,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,532,533,533,535,535,535,535,535,535,535
281,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,11781,11781,11781,11783,11783,11784,11784,11793,11793,11797
282,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,313910,314159,314641,314850,314986,315148,315202,315202,315202,315496


In [3]:
deaths_df


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/7/22,3/8/22,3/9/22,3/10/22,3/11/22,3/12/22,3/13/22,3/14/22,3/15/22,3/16/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,7626,7630,7636,7639,7640,7641,7643,7645,7646,7651
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,3483,3483,3483,3484,3485,3485,3485,3485,3486,3487
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,6857,6858,6860,6861,6861,6862,6864,6866,6868,6869
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,151,152,152,152,152,152,152,152,152,152
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,1900,1900,1900,1900,1900,1900,1900,1900,1900,1900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,5581,5583,5588,5594,5603,5603,5603,5616,5622,5622
280,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
281,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,2138,2138,2138,2139,2139,2139,2139,2139,2139,2140
282,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,3959,3959,3960,3960,3960,3960,3960,3960,3960,3961


In [4]:
recovered_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/7/22,3/8/22,3/9/22,3/10/22,3/11/22,3/12/22,3/13/22,3/14/22,3/15/22,3/16/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
265,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
266,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
267,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#   save DFs to CSVs to do exploratory data analysis
confirmed_df.to_csv('exported_conf_df.csv')
deaths_df.to_csv('exported_deaths_df.csv')
recovered_df.to_csv('exported_recvd_df.csv')


In [6]:
#   use melt() to unpivot DataFrames from current wide format into long format

dates = confirmed_df.columns[4:]
confirmed_df_long = confirmed_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
)
deaths_df_long = deaths_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)
recovered_df_long = recovered_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Recovered'
)
confirmed_df_long

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
222935,NaN,West Bank and Gaza,31.952200,35.233200,3/16/22,654068
222936,NaN,Winter Olympics 2022,39.904200,116.407400,3/16/22,535
222937,NaN,Yemen,15.552727,48.516388,3/16/22,11797
222938,NaN,Zambia,-13.133897,27.849332,3/16/22,315496


In [7]:
deaths_df_long

,Province/State,Country/Region,Lat,Long,Date,Deaths
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
222935,NaN,West Bank and Gaza,31.952200,35.233200,3/16/22,5622
222936,NaN,Winter Olympics 2022,39.904200,116.407400,3/16/22,0
222937,NaN,Yemen,15.552727,48.516388,3/16/22,2140
222938,NaN,Zambia,-13.133897,27.849332,3/16/22,3961


In [8]:
recovered_df_long

,Province/State,Country/Region,Lat,Long,Date,Recovered
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
211160,NaN,West Bank and Gaza,31.952200,35.233200,3/16/22,0
211161,NaN,Winter Olympics 2022,39.904200,116.407400,3/16/22,0
211162,NaN,Yemen,15.552727,48.516388,3/16/22,0
211163,NaN,Zambia,-13.133897,27.849332,3/16/22,0


#### Remove recovered data for Canada due to mismatch issue
*    Canada recovered data is counted for the whole Country instead of by Province/State which is how Canada 
*        and the rest of the world count data for "Confirmed Cases" and "Deaths".

##### We considered apportioning recovered data for the whole country in the same ratio as confirmed cases. 
##### This is arbitrarily altering data from a source and is considered bad practice so we did not do so. 


In [9]:
#   remove Canada recovered data
recovered_df_long = recovered_df_long[recovered_df_long['Country/Region']!='Canada']


####   merge the 3 data frames one after another

In [10]:

#   merge confirmed_df_long and deaths_df_long
full_table = confirmed_df_long.merge(
  right=deaths_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

#    merge full_table and recovered_df_long
full_table = full_table.merge(
  right=recovered_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

full_table


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0,0,0.0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0.0
...,...,...,...,...,...,...,...,...
222935,NaN,West Bank and Gaza,31.952200,35.233200,3/16/22,654068,5622,0.0
222936,NaN,Winter Olympics 2022,39.904200,116.407400,3/16/22,535,0,0.0
222937,NaN,Yemen,15.552727,48.516388,3/16/22,11797,2140,0.0
222938,NaN,Zambia,-13.133897,27.849332,3/16/22,315496,3961,0.0


In [11]:
#   Check Canada data in "full_table" - "recovered" should be 0 and check of CSV file confirms it is 
full_table.to_csv('full_table_can_recvd.csv')

####   Data Cleansing
*    1. converting date from string to datetime
*    2. replacing missing value NaN with zeroes
*    3. coronavirus cases reported from 3 cruise ships should be treated differently

In [12]:
#   1 converting date from string to datetime
full_table['Date'] = pd.to_datetime(full_table['Date'])


In [13]:
full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0
...,...,...,...,...,...,...,...,...
222935,NaN,West Bank and Gaza,31.952200,35.233200,2022-03-16,654068,5622,0.0
222936,NaN,Winter Olympics 2022,39.904200,116.407400,2022-03-16,535,0,0.0
222937,NaN,Yemen,15.552727,48.516388,2022-03-16,11797,2140,0.0
222938,NaN,Zambia,-13.133897,27.849332,2022-03-16,315496,3961,0.0


In [14]:
#   2 replacing missing values NaN
#   detect missing values NaN 
full_table.isna().sum()

Province/State    153075
Country/Region         0
Lat                 1570
Long                1570
Date                   0
Confirmed              0
Deaths                 0
Recovered          16485
dtype: int64

In [15]:
#   replace 'recovered' NaNs with zero
full_table['Recovered'] = full_table['Recovered'].fillna(0)
full_table['Recovered']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
222935    0.0
222936    0.0
222937    0.0
222938    0.0
222939    0.0
Name: Recovered, Length: 222940, dtype: float64

####   3 coronavirus cases reported from 3 cruise ships should be treated differently
*    Cases reported from cruise ships: Grand Princess, Diamond Princess and MS Zaandam need to be extracted and 
*    treated differently due to Province/State and Country/Region mismatch over time

In [16]:
#   extract data for these ships 
ship_rows = full_table['Province/State'].str.contains('Grand Princess') | full_table['Country/Region'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('MS Zaandam')
full_ship = full_table[ship_rows]

#   remove data for these ships 
full_table = full_table[~(ship_rows)]

In [17]:
#   check "full_table" to ensure its integrity
full_table.to_csv('full_table_clean.csv')

#### Calculate the number of Active Cases 
    *Active Cases = Confirmed Cases - Deaths - Recovered Cases 
    

In [18]:
#    calculate active cases = confirmed - deaths - recovered
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']
full_table

<ipython-input-18-9ae382a9de3f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
222935,NaN,West Bank and Gaza,31.952200,35.233200,2022-03-16,654068,5622,0.0,648446.0
222936,NaN,Winter Olympics 2022,39.904200,116.407400,2022-03-16,535,0,0.0,535.0
222937,NaN,Yemen,15.552727,48.516388,2022-03-16,11797,2140,0.0,9657.0
222938,NaN,Zambia,-13.133897,27.849332,2022-03-16,315496,3961,0.0,311535.0


In [19]:
#   aggregate data into Country/Region and group by Date and Country/Region

full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
full_grouped

<ipython-input-19-221a74c92573>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()


,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
154640,2022-03-16,West Bank and Gaza,654068,5622,0.0,648446.0
154641,2022-03-16,Winter Olympics 2022,535,0,0.0,535.0
154642,2022-03-16,Yemen,11797,2140,0.0,9657.0
154643,2022-03-16,Zambia,315496,3961,0.0,311535.0


In [20]:
#   calculate daily New cases, New deaths and New recovered by deducting the corresponding accumulative data on the previous day
#   new cases 
temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan

#   renaming columns
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']

#   merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])

#   filling na with 0
full_grouped = full_grouped.fillna(0)

#   fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')

full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)


<ipython-input-20-0c33f5bb9cb6>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']


####   Final output is data sorted by Date and Country/Region ascending where: -
*    Confirmed Cases, Deaths, Recovered and Active are cumulative data for the entire period, and,
*    New cases, New deaths and New Recovered are daily data

In [21]:
#    Final data frame
full_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
0,2020-01-22,Afghanistan,0,0,0.0,0.0,0,0,0
1,2020-01-22,Albania,0,0,0.0,0.0,0,0,0
2,2020-01-22,Algeria,0,0,0.0,0.0,0,0,0
3,2020-01-22,Andorra,0,0,0.0,0.0,0,0,0
4,2020-01-22,Angola,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...
154640,2022-03-16,West Bank and Gaza,654068,5622,0.0,648446.0,0,0,0
154641,2022-03-16,Winter Olympics 2022,535,0,0.0,535.0,0,0,0
154642,2022-03-16,Yemen,11797,2140,0.0,9657.0,4,1,0
154643,2022-03-16,Zambia,315496,3961,0.0,311535.0,294,1,0


In [23]:
#   convert dataframe to a csv file for backup
# full_grouped.to_csv('exported_CSVs/covid_cases.csv')


#### Check data for Australia against JHU Daily Replorts to confirm that it is correct 
#### Data is correct as at 07/02/2022 - 2,727,260 confirmed cases & 4,200 confirmed deaths


In [24]:
#   select Oz to check that data is correct
full_grouped_oz = full_grouped.loc[full_grouped["Country/Region"] == "Australia"]
full_grouped_oz

# Check data for Australia against JHU Daily Replorts to confirm that it is correct. 
# Data is correct as at 07/02/2022 - 2,727,260 confirmed cases & 4,200 confirmed deaths

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
9,2020-01-22,Australia,0,0,0.0,0.0,0,0,0
206,2020-01-23,Australia,0,0,0.0,0.0,0,0,0
403,2020-01-24,Australia,0,0,0.0,0.0,0,0,0
600,2020-01-25,Australia,0,0,0.0,0.0,0,0,0
797,2020-01-26,Australia,4,0,0.0,4.0,4,0,0
...,...,...,...,...,...,...,...,...,...
153669,2022-03-12,Australia,3596466,5587,0.0,3590879.0,33495,16,0
153866,2022-03-13,Australia,3622906,5590,0.0,3617316.0,26440,3,0
154063,2022-03-14,Australia,3651709,5615,0.0,3646094.0,28803,25,0
154260,2022-03-15,Australia,3706018,5638,0.0,3700380.0,54309,23,0


#### VACCINATION DATASET SOURCED FROM OUR WORLD IN DATA AT THE FOLLOWING LINK
*    https://ourworldindata.org/covid-vaccinations


In [28]:
#   read the vaccination dataset - csv file into a dataframe
vacc_data_df = pd.read_csv('owid-covid-data.csv')
vacc_data_df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159776,ZWE,Africa,Zimbabwe,2022-02-01,229851.0,185.0,153.571,5350.0,12.0,4.857,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
159777,ZWE,Africa,Zimbabwe,2022-02-02,230012.0,161.0,152.714,5352.0,2.0,4.429,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
159778,ZWE,Africa,Zimbabwe,2022-02-03,230170.0,158.0,153.429,5357.0,5.0,4.714,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
159779,ZWE,Africa,Zimbabwe,2022-02-04,230170.0,0.0,119.571,5357.0,0.0,3.429,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN


#### Create data frame with just people vaccinated data 


In [29]:
vacc_data_df = vacc_data_df[["iso_code", "continent", "location", "date", "people_vaccinated_per_hundred",
                             "people_fully_vaccinated_per_hundred", "total_boosters_per_hundred"]]
vacc_data_df


,iso_code,continent,location,date,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred
0,AFG,Asia,Afghanistan,2020-02-24,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,NaN,NaN,NaN
...,...,...,...,...,...,...,...
159776,ZWE,Africa,Zimbabwe,2022-02-01,28.32,21.89,NaN
159777,ZWE,Africa,Zimbabwe,2022-02-02,28.38,21.94,NaN
159778,ZWE,Africa,Zimbabwe,2022-02-03,28.40,21.96,NaN
159779,ZWE,Africa,Zimbabwe,2022-02-04,28.44,22.00,NaN


In [30]:
#   detect missing values NaN 
vacc_data_df.isna().sum()


iso_code                                    0
continent                                9592
location                                    0
date                                        0
people_vaccinated_per_hundred          119200
people_fully_vaccinated_per_hundred    122094
total_boosters_per_hundred             144784
dtype: int64

In [31]:
#   replace NaNs with zero
vacc_data_df['people_vaccinated_per_hundred'] = vacc_data_df['people_vaccinated_per_hundred'].fillna(0)
vacc_data_df['people_fully_vaccinated_per_hundred'] = vacc_data_df['people_vaccinated_per_hundred'].fillna(0)
vacc_data_df['total_boosters_per_hundred'] = vacc_data_df['total_boosters_per_hundred'].fillna(0)
vacc_data_df


<ipython-input-31-c1a4dbe2fd07>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacc_data_df['people_vaccinated_per_hundred'] = vacc_data_df['people_vaccinated_per_hundred'].fillna(0)
<ipython-input-31-c1a4dbe2fd07>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacc_data_df['people_fully_vaccinated_per_hundred'] = vacc_data_df['people_vaccinated_per_hundred'].fillna(0)
<ipython-input-31-c1a4dbe2fd07>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,iso_code,continent,location,date,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred
0,AFG,Asia,Afghanistan,2020-02-24,0.00,0.00,0.0
1,AFG,Asia,Afghanistan,2020-02-25,0.00,0.00,0.0
2,AFG,Asia,Afghanistan,2020-02-26,0.00,0.00,0.0
3,AFG,Asia,Afghanistan,2020-02-27,0.00,0.00,0.0
4,AFG,Asia,Afghanistan,2020-02-28,0.00,0.00,0.0
...,...,...,...,...,...,...,...
159776,ZWE,Africa,Zimbabwe,2022-02-01,28.32,28.32,0.0
159777,ZWE,Africa,Zimbabwe,2022-02-02,28.38,28.38,0.0
159778,ZWE,Africa,Zimbabwe,2022-02-03,28.40,28.40,0.0
159779,ZWE,Africa,Zimbabwe,2022-02-04,28.44,28.44,0.0


In [32]:
#   data cleansing'Country/Region].replace(['United States'], 'US')
vacc_data_df['location'] = vacc_data_df['location'].replace(['United States'],'US')
vacc_data_df


<ipython-input-32-72db0ae0a4c0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacc_data_df['location'] = vacc_data_df['location'].replace(['United States'],'US')


,iso_code,continent,location,date,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred
0,AFG,Asia,Afghanistan,2020-02-24,0.00,0.00,0.0
1,AFG,Asia,Afghanistan,2020-02-25,0.00,0.00,0.0
2,AFG,Asia,Afghanistan,2020-02-26,0.00,0.00,0.0
3,AFG,Asia,Afghanistan,2020-02-27,0.00,0.00,0.0
4,AFG,Asia,Afghanistan,2020-02-28,0.00,0.00,0.0
...,...,...,...,...,...,...,...
159776,ZWE,Africa,Zimbabwe,2022-02-01,28.32,28.32,0.0
159777,ZWE,Africa,Zimbabwe,2022-02-02,28.38,28.38,0.0
159778,ZWE,Africa,Zimbabwe,2022-02-03,28.40,28.40,0.0
159779,ZWE,Africa,Zimbabwe,2022-02-04,28.44,28.44,0.0


#### people_vaccinated_per_hundred is non zero if people have had one vaccination, so we can calculate the 
#### unvaccinated as follows: people_not_vaccinated_per_hundred = 100 - people_vaccinated_per_hundred.


In [33]:
#   now calculate people_not_vaccinated_per_hundred = 100 - people_vaccinated_per_hundred 
vacc_data_df['people_not_vaccinated_per_hundred'] = 0
vacc_data_df['people_not_vaccinated_per_hundred'] = np.where(vacc_data_df['people_vaccinated_per_hundred'] != 0,
                        100 - vacc_data_df['people_vaccinated_per_hundred'], vacc_data_df['people_not_vaccinated_per_hundred'])



<ipython-input-33-c992f0f7e920>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacc_data_df['people_not_vaccinated_per_hundred'] = 0
<ipython-input-33-c992f0f7e920>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacc_data_df['people_not_vaccinated_per_hundred'] = np.where(vacc_data_df['people_vaccinated_per_hundred'] != 0,


In [34]:
vacc_data_df

,iso_code,continent,location,date,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,people_not_vaccinated_per_hundred
0,AFG,Asia,Afghanistan,2020-02-24,0.00,0.00,0.0,0.00
1,AFG,Asia,Afghanistan,2020-02-25,0.00,0.00,0.0,0.00
2,AFG,Asia,Afghanistan,2020-02-26,0.00,0.00,0.0,0.00
3,AFG,Asia,Afghanistan,2020-02-27,0.00,0.00,0.0,0.00
4,AFG,Asia,Afghanistan,2020-02-28,0.00,0.00,0.0,0.00
...,...,...,...,...,...,...,...,...
159776,ZWE,Africa,Zimbabwe,2022-02-01,28.32,28.32,0.0,71.68
159777,ZWE,Africa,Zimbabwe,2022-02-02,28.38,28.38,0.0,71.62
159778,ZWE,Africa,Zimbabwe,2022-02-03,28.40,28.40,0.0,71.60
159779,ZWE,Africa,Zimbabwe,2022-02-04,28.44,28.44,0.0,71.56


In [36]:
# save cleansed vaccination data to a CSV for backup
# vacc_data_df.to_csv('exported_CSVs/vaccinations.csv')


#### WORLD POPULATION DATASET AS AT 31/12/2020 SOURCED FROM THE FOLLOWING LINK
*    https://www.worldometers.info/world-population/population-by-country/ 

In [37]:
#   read the world population dataset - csv file into a dataframe
pop_data_df = pd.read_csv('world_population.csv')
pop_data_df


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54208.0,55434.0,56234.0,56699.0,57029.0,57357.0,...,102050.0,102565.0,103165.0,103776.0,104339.0,104865.0,105361.0,105846.0,106310.0,106766.0
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130836765.0,134159786.0,137614644.0,141202036.0,144920186.0,148769974.0,...,532760424.0,547482863.0,562601578.0,578075373.0,593871847.0,609978946.0,626392880.0,643090131.0,660046272.0,677243299.0
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,...,30117411.0,31161378.0,32269592.0,33370804.0,34413603.0,35383028.0,36296111.0,37171922.0,38041757.0,38928341.0
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,96396419.0,98407221.0,100506960.0,102691339.0,104953470.0,107289875.0,...,360285439.0,370243017.0,380437896.0,390882979.0,401586651.0,412551299.0,423769930.0,435229381.0,446911598.0,458803476.0
4,Angola,AGO,"Population, total",SP.POP.TOTL,5454938.0,5531451.0,5608499.0,5679409.0,5734995.0,5770573.0,...,24220660.0,25107925.0,26015786.0,26941773.0,27884380.0,28842482.0,29816769.0,30809787.0,31825299.0,32866268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,"Population, total",SP.POP.TOTL,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,...,1791000.0,1807106.0,1818117.0,1812771.0,1788196.0,1777557.0,1791003.0,1797085.0,1788878.0,1775378.0
262,"Yemen, Rep.",YEM,"Population, total",SP.POP.TOTL,5315351.0,5393034.0,5473671.0,5556767.0,5641598.0,5727745.0,...,23807586.0,24473176.0,25147112.0,25823488.0,26497881.0,27168210.0,27834811.0,28498683.0,29161922.0,29825968.0
263,South Africa,ZAF,"Population, total",SP.POP.TOTL,17099836.0,17524533.0,17965733.0,18423157.0,18896303.0,19384838.0,...,52003759.0,52832659.0,53687125.0,54544184.0,55386369.0,56207649.0,57009751.0,57792520.0,58558267.0,59308690.0
264,Zambia,ZMB,"Population, total",SP.POP.TOTL,3070780.0,3164330.0,3260645.0,3360099.0,3463211.0,3570466.0,...,14023199.0,14465148.0,14926551.0,15399793.0,15879370.0,16363449.0,16853608.0,17351714.0,17861034.0,18383956.0


In [38]:
#   detect missing values NaN 
pop_data_df.isna().sum()


Country Name      0
Country Code      0
Indicator Name    0
Indicator Code    0
1960              2
                 ..
2016              2
2017              2
2018              2
2019              2
2020              2
Length: 65, dtype: int64

In [39]:
#   replace NaNs with zero
pop_data_df['1960'] = pop_data_df['1960'].fillna(0)
pop_data_df['2016'] = pop_data_df['2016'].fillna(0)
pop_data_df['2017'] = pop_data_df['2017'].fillna(0)
pop_data_df['2018'] = pop_data_df['2018'].fillna(0)
pop_data_df['2019'] = pop_data_df['2019'].fillna(0)
pop_data_df['2020'] = pop_data_df['2020'].fillna(0)
pop_data_df

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54208.0,55434.0,56234.0,56699.0,57029.0,57357.0,...,102050.0,102565.0,103165.0,103776.0,104339.0,104865.0,105361.0,105846.0,106310.0,106766.0
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130836765.0,134159786.0,137614644.0,141202036.0,144920186.0,148769974.0,...,532760424.0,547482863.0,562601578.0,578075373.0,593871847.0,609978946.0,626392880.0,643090131.0,660046272.0,677243299.0
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,...,30117411.0,31161378.0,32269592.0,33370804.0,34413603.0,35383028.0,36296111.0,37171922.0,38041757.0,38928341.0
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,96396419.0,98407221.0,100506960.0,102691339.0,104953470.0,107289875.0,...,360285439.0,370243017.0,380437896.0,390882979.0,401586651.0,412551299.0,423769930.0,435229381.0,446911598.0,458803476.0
4,Angola,AGO,"Population, total",SP.POP.TOTL,5454938.0,5531451.0,5608499.0,5679409.0,5734995.0,5770573.0,...,24220660.0,25107925.0,26015786.0,26941773.0,27884380.0,28842482.0,29816769.0,30809787.0,31825299.0,32866268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,"Population, total",SP.POP.TOTL,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,...,1791000.0,1807106.0,1818117.0,1812771.0,1788196.0,1777557.0,1791003.0,1797085.0,1788878.0,1775378.0
262,"Yemen, Rep.",YEM,"Population, total",SP.POP.TOTL,5315351.0,5393034.0,5473671.0,5556767.0,5641598.0,5727745.0,...,23807586.0,24473176.0,25147112.0,25823488.0,26497881.0,27168210.0,27834811.0,28498683.0,29161922.0,29825968.0
263,South Africa,ZAF,"Population, total",SP.POP.TOTL,17099836.0,17524533.0,17965733.0,18423157.0,18896303.0,19384838.0,...,52003759.0,52832659.0,53687125.0,54544184.0,55386369.0,56207649.0,57009751.0,57792520.0,58558267.0,59308690.0
264,Zambia,ZMB,"Population, total",SP.POP.TOTL,3070780.0,3164330.0,3260645.0,3360099.0,3463211.0,3570466.0,...,14023199.0,14465148.0,14926551.0,15399793.0,15879370.0,16363449.0,16853608.0,17351714.0,17861034.0,18383956.0


In [41]:
# save cleansed population data to a CSV for backup
# pop_data_df.to_csv('exported_CSVs/world_population.csv')

#### Create dataframe for Country Codes table

In [42]:
# Copy OWID Vaccination dataframe, as we want to use OWID country codes
country_codes = vacc_data_df[["iso_code","location","continent"]]
country_codes_cleaned = country_codes.drop_duplicates(subset=["iso_code"])
country_codes = country_codes_cleaned.rename(columns={"iso_code":"country_id",
                                              "location":"country_name",
                                              "continent":"continent_name"})


In [43]:
#Adding countries to country code dataframe to match population dataframe
africa_eastern = {'country_id': 'AFE', 'country_name': 'Africa Eastern and Southern', 'continent_name': 'Africa'}
africa_western = {'country_id': 'AFW', 'country_name': 'Africa Western and Central', 'continent_name': 'Africa'}
arab_world = {'country_id': 'ARB', 'country_name': 'Arab World', 'continent_name': 'Asia'}
asm = {'country_id': 'ASM', 'country_name': 'American Samoa', 'continent_name': 'Oceania'}
chi = {'country_id': 'CHI', 'country_name': 'Channel Islands', 'continent_name': 'Europe'}
gum = {'country_id': 'GUM', 'country_name': 'Guam', 'continent_name': 'Oceania'}
mnp = {'country_id': 'MNP', 'country_name': 'Northern Mariana Islands', 'continent_name': 'Oceania'}
pri = {'country_id': 'PRI', 'country_name': 'Puerto Rico', 'continent_name': 'North America'}
prk = {'country_id': 'PRK', 'country_name': 'Korea, Dem. Peoples Rep.', 'continent_name': 'Asia'}
vir = {'country_id': 'VIR', 'country_name': 'Virgin Islands (U.S.)', 'continent_name': 'North America'}
xkx = {'country_id': 'XKX', 'country_name': 'Kosovo', 'continent_name': 'Europe'}
maf = {'country_id': 'MAF', 'country_name': 'St. Martin', 'continent_name': 'North America'}

country_codes = country_codes.append(africa_eastern, ignore_index = True)
country_codes = country_codes.append(africa_western, ignore_index = True)
country_codes = country_codes.append(arab_world, ignore_index = True)
country_codes = country_codes.append(asm, ignore_index = True)
country_codes = country_codes.append(chi, ignore_index = True)
country_codes = country_codes.append(gum, ignore_index = True)
country_codes = country_codes.append(mnp, ignore_index = True)
country_codes = country_codes.append(pri, ignore_index = True)
country_codes = country_codes.append(prk, ignore_index = True)
country_codes = country_codes.append(vir, ignore_index = True)
country_codes = country_codes.append(xkx, ignore_index = True)
country_codes = country_codes.append(maf, ignore_index = True)

country_codes


,country_id,country_name,continent_name
0,AFG,Afghanistan,Asia
1,OWID_AFR,Africa,NaN
2,ALB,Albania,Europe
3,DZA,Algeria,Africa
4,AND,Andorra,Europe
...,...,...,...
245,PRI,Puerto Rico,North America
246,PRK,"Korea, Dem. Peoples Rep.",Asia
247,VIR,Virgin Islands (U.S.),North America
248,XKX,Kosovo,Europe


#### Edit full_grouped covid case dataframe to include country ID

In [64]:
full_grouped_copy = full_grouped.copy()
full_grouped_copy = full_grouped_copy.rename(columns={"Country/Region":"country_name"})
full_grouped_reformatted = pd.merge(country_codes,full_grouped_copy,on="country_name")

,country_id,country_name,continent_name,Date,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
0,AFG,Afghanistan,Asia,2020-01-22,0,0,0.0,0.0,0,0,0
1,AFG,Afghanistan,Asia,2020-01-23,0,0,0.0,0.0,0,0,0
2,AFG,Afghanistan,Asia,2020-01-24,0,0,0.0,0.0,0,0,0
3,AFG,Afghanistan,Asia,2020-01-25,0,0,0.0,0.0,0,0,0
4,AFG,Afghanistan,Asia,2020-01-26,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
144435,ZWE,Zimbabwe,Africa,2022-03-12,242069,5412,0.0,236657.0,521,4,0
144436,ZWE,Zimbabwe,Africa,2022-03-13,242515,5414,0.0,237101.0,446,2,0
144437,ZWE,Zimbabwe,Africa,2022-03-14,242909,5416,0.0,237493.0,394,2,0
144438,ZWE,Zimbabwe,Africa,2022-03-15,242909,5416,0.0,237493.0,0,0,0


### Change structure of dataframes to match structure of tables created in the database

In [45]:
# Covid Cases table
# Copy only the columns needed into a new DataFrame.
covid_cases = full_grouped_reformatted[["country_id","Date","Confirmed","Deaths","Recovered","Active","New cases","New deaths","New recovered"]]

# Rename columns to fit the tables created in the database.
covid_cases = covid_cases.rename(columns={"Date":"date",
                                          "Confirmed":"confirmed",
                                          "Deaths":"deaths",
                                          "Recovered":"recovered",
                                          "Active":"active",
                                          "New cases":"new_cases",
                                          "New deaths":"new_deaths",
                                          "New recovered":"new_recovered"})
covid_cases


,country_id,date,confirmed,deaths,recovered,active,new_cases,new_deaths,new_recovered
0,AFG,2020-01-22,0,0,0.0,0.0,0,0,0
1,AFG,2020-01-23,0,0,0.0,0.0,0,0,0
2,AFG,2020-01-24,0,0,0.0,0.0,0,0,0
3,AFG,2020-01-25,0,0,0.0,0.0,0,0,0
4,AFG,2020-01-26,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...
144435,ZWE,2022-03-12,242069,5412,0.0,236657.0,521,4,0
144436,ZWE,2022-03-13,242515,5414,0.0,237101.0,446,2,0
144437,ZWE,2022-03-14,242909,5416,0.0,237493.0,394,2,0
144438,ZWE,2022-03-15,242909,5416,0.0,237493.0,0,0,0


In [46]:
# Population table
# Copy only the columns needed into a new DataFrame.
population = pop_data_df[["Country Code","2020"]]

# Remove country codes that aren't actually countries
population = population[population["Country Code"].str.contains('CEB')==False]
population = population[population["Country Code"].str.contains('Country Code')==False]
population = population[population["Country Code"].str.contains('CSS')==False]
population = population[population["Country Code"].str.contains('EAP')==False]
population = population[population["Country Code"].str.contains('EAR')==False]
population = population[population["Country Code"].str.contains('EAS')==False]
population = population[population["Country Code"].str.contains('ECA')==False]
population = population[population["Country Code"].str.contains('ECS')==False]
population = population[population["Country Code"].str.contains('EMU')==False]
population = population[population["Country Code"].str.contains('EUU')==False]
population = population[population["Country Code"].str.contains('FCS')==False]
population = population[population["Country Code"].str.contains('HIC')==False]
population = population[population["Country Code"].str.contains('HPC')==False]
population = population[population["Country Code"].str.contains('IBD')==False]
population = population[population["Country Code"].str.contains('IBT')==False]
population = population[population["Country Code"].str.contains('IDA')==False]
population = population[population["Country Code"].str.contains('IDB')==False]
population = population[population["Country Code"].str.contains('IDX')==False]
population = population[population["Country Code"].str.contains('INX')==False]
population = population[population["Country Code"].str.contains('LAC')==False]
population = population[population["Country Code"].str.contains('LCN')==False]
population = population[population["Country Code"].str.contains('LDC')==False]
population = population[population["Country Code"].str.contains('LIC')==False]
population = population[population["Country Code"].str.contains('LMC')==False]
population = population[population["Country Code"].str.contains('LMY')==False]
population = population[population["Country Code"].str.contains('LTE')==False]
population = population[population["Country Code"].str.contains('MEA')==False]
population = population[population["Country Code"].str.contains('MIC')==False]
population = population[population["Country Code"].str.contains('MNA')==False]
population = population[population["Country Code"].str.contains('NAC')==False]
population = population[population["Country Code"].str.contains('OED')==False]
population = population[population["Country Code"].str.contains('OSS')==False]
population = population[population["Country Code"].str.contains('PRE')==False]
population = population[population["Country Code"].str.contains('PSS')==False]
population = population[population["Country Code"].str.contains('PST')==False]
population = population[population["Country Code"].str.contains('SAS')==False]
population = population[population["Country Code"].str.contains('SSA')==False]
population = population[population["Country Code"].str.contains('SSF')==False]
population = population[population["Country Code"].str.contains('SST')==False]
population = population[population["Country Code"].str.contains('TEA')==False]
population = population[population["Country Code"].str.contains('TEC')==False]
population = population[population["Country Code"].str.contains('TLA')==False]
population = population[population["Country Code"].str.contains('TMN')==False]
population = population[population["Country Code"].str.contains('TSA')==False]
population = population[population["Country Code"].str.contains('TSS')==False]
population = population[population["Country Code"].str.contains('UMC')==False]
population = population[population["Country Code"].str.contains('WLD')==False]

# Rename columns to fit the tables created in the database.
population = population.rename(columns={"Country Code":"country_id",
                                          "2020":"population"})
population


,country_id,population
0,ABW,106766.0
1,AFE,677243299.0
2,AFG,38928341.0
3,AFW,458803476.0
4,AGO,32866268.0
...,...,...
261,XKX,1775378.0
262,YEM,29825968.0
263,ZAF,59308690.0
264,ZMB,18383956.0


In [47]:
# Vaccinations table
# Copy only the columns needed into a new DataFrame.
vaccinations = vacc_data_df[["iso_code","date","people_vaccinated_per_hundred", "people_fully_vaccinated_per_hundred",
                             "total_boosters_per_hundred", "people_not_vaccinated_per_hundred"]]

# Rename columns to fit the tables created in the database.
vaccinations = vaccinations.rename(columns={"iso_code":"country_id",
                                          "people_vaccinated_per_hundred":"vaccinated_per_hundred",
                                          "people_fully_vaccinated_per_hundred":"fully_vaccinated_per_hundred",
                                          "total_boosters_per_hundred":"boosted_per_hundred",
                                          "people_not_vaccinated_per_hundred":"not_fully_vaccinated_per_hundred"})
vaccinations


,country_id,date,vaccinated_per_hundred,fully_vaccinated_per_hundred,boosted_per_hundred,not_fully_vaccinated_per_hundred
0,AFG,2020-02-24,0.00,0.00,0.0,0.00
1,AFG,2020-02-25,0.00,0.00,0.0,0.00
2,AFG,2020-02-26,0.00,0.00,0.0,0.00
3,AFG,2020-02-27,0.00,0.00,0.0,0.00
4,AFG,2020-02-28,0.00,0.00,0.0,0.00
...,...,...,...,...,...,...
159776,ZWE,2022-02-01,28.32,28.32,0.0,71.68
159777,ZWE,2022-02-02,28.38,28.38,0.0,71.62
159778,ZWE,2022-02-03,28.40,28.40,0.0,71.60
159779,ZWE,2022-02-04,28.44,28.44,0.0,71.56


In [80]:
# Change data type of vaccination dates to datetime
vaccinations["date"] = pd.to_datetime(vaccinations["date"])
covid_cases["date"] = pd.to_datetime(covid_cases["date"])

In [81]:
# Create full table with merged information
full_covid_table_1 = pd.merge(covid_cases, vaccinations,  how='outer', on=['country_id','date'])
full_covid_table_2 = pd.merge(full_covid_table_1, country_codes, how='outer', on = 'country_id')
full_covid_table_3 = pd.merge(full_covid_table_2, population, how='outer', on = 'country_id')
full_covid_table = full_covid_table_3[["country_id","country_name","continent_name","population","date","confirmed","deaths","recovered","active","new_cases","new_deaths","new_recovered","vaccinated_per_hundred","fully_vaccinated_per_hundred","boosted_per_hundred","not_fully_vaccinated_per_hundred"]]
full_covid_table = full_covid_table.fillna(0)
full_covid_table

,country_id,country_name,continent_name,population,date,confirmed,deaths,recovered,active,new_cases,new_deaths,new_recovered,vaccinated_per_hundred,fully_vaccinated_per_hundred,boosted_per_hundred,not_fully_vaccinated_per_hundred
0,AFG,Afghanistan,Asia,38928341.0,2020-01-22 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AFG,Afghanistan,Asia,38928341.0,2020-01-23 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AFG,Afghanistan,Asia,38928341.0,2020-01-24 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AFG,Afghanistan,Asia,38928341.0,2020-01-25 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AFG,Afghanistan,Asia,38928341.0,2020-01-26 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177465,MNP,Northern Mariana Islands,Oceania,57557.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
177466,PRI,Puerto Rico,North America,3194034.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
177467,PRK,"Korea, Dem. Peoples Rep.",Asia,25778815.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
177468,VIR,Virgin Islands (U.S.),North America,106290.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
# Save all tables to CSVs for use in Javascript
country_codes.to_csv('../tables/country_codes.csv')
covid_cases.to_csv('../tables/covid_cases.csv')
population.to_csv('../tables/population.csv')
vaccinations.to_csv('../tables/vaccinations.csv')
full_covid_table.to_csv('../tables/full_covid_table.csv')

### Load tables to Integrated Covid View database

In [83]:
# Create database connection
rds_connection_string = "postgres:meg221196@localhost:5432/integrated_covid_view_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

# Confirm database tables
engine.table_names()

<ipython-input-83-9ccd88317c4f>:6: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['country_codes',
 'covid_cases',
 'population',
 'vaccinations',
 'full_covid_table']

In [84]:
# Load dataframes to database tables
full_covid_table.to_sql(name='full_covid_table', con=engine, if_exists='append', index=False)
# covid_cases.to_sql(name='covid_cases', con=engine, if_exists='append', index=False)
# population.to_sql(name='population', con=engine, if_exists='append', index=False)
# vaccinations.to_sql(name='vaccinations', con=engine, if_exists='append', index=False)